In [150]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [151]:

time = '2025-07-01'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df_all = results.to_dataframe()
# add index
df_all['index'] = df_all.index
df_all['event_time_temp'] = df_all['event_time'].dt.strftime('%Y-%m-%d %H:%M')
df_all['event_time'] = df_all['event_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df_all.drop_duplicates(subset=['sso_id', 'campaign_details','event_time'], keep='last')
response = df[df['campaign_name'] == 'response answer'].drop_duplicates(subset=['sso_id','campaign_details'], keep='last')
df_f = pd.concat([df[df['campaign_name'] == 'open question'], response], ignore_index=True)
df_f = df_f.drop(columns=['event_time_temp'])
print(df_f.shape[0])
df_f = df_f.reset_index(drop=True)
df = df_f.copy()
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



2639


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index
0,8853972261,"[{'key': 'session_id', 'value': '8cf8bf48-8148...",ראיתי כבר,open question,App,2025-07-01 13:49:14,action,0
1,286836061038,"[{'key': 'session_id', 'value': 'c7df777b-13aa...",יש משהו סובטילי בנטפליקס?,open question,App,2025-07-01 13:49:06,action,2


In [152]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
# add index to parsed data
expanded_df = pd.json_normalize(df['parsed_data'].tolist())

expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,session_id,total_time,rag_speed,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,8853972261,ראיתי כבר,open question,App,2025-07-01 13:49:14,action,0,8cf8bf48-8148-426c-a976-ab48004aa0d1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8853972261_8cf8bf48-8148-426c-a976-ab48004aa0d1
1,286836061038,יש משהו סובטילי בנטפליקס?,open question,App,2025-07-01 13:49:06,action,2,c7df777b-13aa-4cf6-9723-0d69108bcaca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286836061038_c7df777b-13aa-4cf6-9723-0d69108bcaca


In [153]:
df['id'].value_counts().sort_values(ascending=False).shape

(596,)

In [154]:
df_all[df_all['sso_id'] == 3057469657]

,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index,event_time_temp
635,3057469657,"[{'key': 'total_time', 'value': '6.96076798439...","אני מבין שאתם מחפשים סדרת פשע בנטפליקס, אבל אי...",response answer,App,2025-07-01 12:44:27,impression,635,2025-07-01 12:44
641,3057469657,"[{'key': 'total_time', 'value': '6.96076798439...","אני מבין שאתם מחפשים סדרת פשע בנטפליקס, אבל אי...",response answer,App,2025-07-01 12:44:22,impression,641,2025-07-01 12:44
645,3057469657,"[{'key': 'session_id', 'value': '34a4cfda-3468...",תביא סדרת פשע חדשה בנטפליקס,open question,App,2025-07-01 12:44:10,action,645,2025-07-01 12:44


In [155]:
def parse_conversation(ids:str,time:bool) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            if time:
                text = f"{row['event_time']} - USER: {row['campaign_details']}"
            else:
                text = f"USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            if time:
                text = f"{row['event_time']} - LLM: {row['campaign_details']}"
            else:
                text = f"LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('8012159237_37b7f6b3-c625-4bb8-a427-b71639d96e3c', True)
print(txt)

2025-07-01 10:04:36 - USER: מה אתה לובש
2025-07-01 10:04:40 - LLM: אני מודל שפה גדול, אין לי מה ללבוש! אבל אני בהחלט יכול לעזור לכם למצוא סרט או סדרה נהדרים לצפייה. במה תרצו שאתמקד? האם תרצו שאמליץ לכם על סרט או סדרה בז'אנר מסוים, או שאולי אתם מחפשים משהו לצפות בו בפלטפורמה ספציפית?
2025-07-01 10:04:52 - USER: כמה סרטים יש סהכ?
2025-07-01 10:04:54 - LLM: מצטער, איני יכול לעזור עם בקשה זו. אני יכול להמליץ לכם על סרטים וסדרות על בסיס ז'אנר, פלטפורמת סטרימינג או תיאור. האם תרצו המלצה על סרט או סדרה כלשהי? האם תרצו שאמליץ לכם על סרט או סדרה בז'אנר מסוים, או שאולי אתם מחפשים משהו לצפות בו בפלטפורמה ספציפית?
2025-07-01 10:05:14 - USER: כמה סרטים אתה מכיר שמתחילים בthe
2025-07-01 10:05:17 - LLM: מצטער, איני יכול לעזור עם בקשה זו. היכולות שלי מתמקדות במציאת המלצות לסרטים וסדרות על פי ז'אנר, פלטפורמת סטרימינג או תיאור. האם תרצו שאמליץ לכם על סרט או סדרה כלשהי? האם תרצו שאמליץ לכם על סרט או סדרה בז'אנר מסוים, או שאולי אתם מחפשים משהו לצפות בו בפלטפורמה ספציפית?
2025-07-01 10:05:35 - USER: תוכל ל

In [156]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    # ---- quality -----
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    politeness: int = Field(..., description="Politeness rating of the LLM response, from 1 to 5, where 1 is rude and 5 is polite.")
    knowledge_from_RAG: bool = Field(..., description="Whether the LLM response used knowledge from RAG (True or False).")
    answer_relevance: int = Field(..., description="Relevance of the LLM response to the user's question, from 1 to 5, where 1 is not relevant and 5 is highly relevant.")
    # ---- LLM beahavior -----
    not_reveal_system_info: bool = Field(..., description="Whether the LLM response did not reveal any system information (True or False).")
    # prompt_enjected: bool = Field(..., description="Whether the LLM response was prompted to inject system information (True or False).")
    # ---- LLM breakdown -----
    hallucination: bool = Field(..., description="Whether the LLM response contained hallucinations (True or False). for ")

In [157]:
from google import genai
import os

model_name = "gemini-2.5-flash"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
# temp = client.models.generate_content(
#     model=model_name,
#     contents=txt,
#                 config={
#             'response_mime_type': 'application/json',
#             'response_schema': ReviewChat,
#             'system_instruction': system_instruction,
#         },
# )



In [158]:
# temp.parsed

In [159]:
# temp.parsed.feedback

# Dashboard

In [160]:
# Calculate text lengths
df['question_length'] = df[df['campaign_name'] == 'open question']['campaign_details'].str.split().str.len()
df['answer_length'] = df[df['campaign_name'] == 'response answer']['campaign_details'].str.split().str.len()

# Create separate dataframes for questions and answers
questions_df = df[df['campaign_name'] == 'open question'].copy()
answers_df = df[df['campaign_name'] == 'response answer'].copy()

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('התפלגות אורך שאלות משתמשים', 'התפלגות אורך תשובות המודל',
                   'Histogram - אורך שאלות', 'Histogram - אורך תשובות'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Box plot for questions
fig.add_trace(
    go.Box(y=questions_df['question_length'].dropna(), 
           name='אורך שאלות',
           boxpoints='outliers'),
    row=1, col=1
)

# Box plot for answers  
fig.add_trace(
    go.Box(y=answers_df['answer_length'].dropna(),
           name='אורך תשובות',
           boxpoints='outliers'),
    row=1, col=2
)

# Histogram for questions
fig.add_trace(
    go.Histogram(x=questions_df['question_length'].dropna(),
                name='שאלות',
                nbinsx=30,
                opacity=0.7),
    row=2, col=1
)

# Histogram for answers
fig.add_trace(
    go.Histogram(x=answers_df['answer_length'].dropna(),
                name='תשובות', 
                nbinsx=30,
                opacity=0.7),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="התפלגות אורך טקסט - שאלות ותשובות",
    title_x=0.5,
    showlegend=False
)

# Update axes labels
fig.update_xaxes(title_text="אורך תווים", row=2, col=1)
fig.update_xaxes(title_text="אורך תווים", row=2, col=2)
fig.update_yaxes(title_text="אורך תווים", row=1, col=1)
fig.update_yaxes(title_text="אורך תווים", row=1, col=2)
fig.update_yaxes(title_text="תדירות", row=2, col=1)
fig.update_yaxes(title_text="תדירות", row=2, col=2)

fig.show()






In [161]:
df[df['question_length'] == df['question_length'].max()]['campaign_details'].values[0]

'יש לי נגישות לערוצי יס אבל אני יכול למצוא לבד סדרות ברשת בהתבסס על כך שאני אובה סדרות עם משמעות, מסתורין, מקוריות ולא קיטשיות או רומנטיות, המלץ על סדרה מחמש השנים האחרונות, שלא בוטלה אחרי עונה אחת או שתיים וזוכה לביקורת טובה, מעל ארבעה כוכבים.'

In [162]:
platform_counts = df['platform'].value_counts()
fig_platform = px.pie(
    values=platform_counts.values, 
    names=platform_counts.index,
    title="התפלגות לפי פלטפורמה"
)
fig_platform.show()

df['hour'] = pd.to_datetime(df['event_time']).dt.hour
hourly_activity = df['hour'].value_counts().sort_index()

fig_hourly = px.bar(
    x=hourly_activity.index,
    y=hourly_activity.values,
    title="פעילות לפי שעה ביום",
    labels={'x': 'שעה', 'y': 'מספר הודעות'}
)
fig_hourly.show()

user_activity = df['sso_id'].value_counts()
fig_user_activity = px.histogram(
    x=user_activity.values,
    nbins=20,
    title="התפלגות משתמשים לפי כמות הודעות",
    labels={'x': 'מספר הודעות', 'y': 'מספר משתמשים'}
)
fig_user_activity.show()

questions_with_answers = df[df['campaign_name'] == 'open question']['question_length'].dropna()
answers_with_questions = df[df['campaign_name'] == 'response answer']['answer_length'].dropna()

if len(questions_with_answers) > 0 and len(answers_with_questions) > 0:
    fig_comparison = go.Figure()
    fig_comparison.add_trace(go.Scatter(
        x=questions_with_answers.values,
        y=answers_with_questions.values,
        mode='markers',
        name='אורך תשובה vs שאלה',
        opacity=0.6
    ))

# 5. מספר הודעות לפי משתמש (Top 10)
top_users = df['sso_id'].value_counts().head(10)
fig_top_users = px.bar(
    x=top_users.index.astype(str),
    y=top_users.values,
    title="10 המשתמשים הכי פעילים",
    labels={'x': 'SSO ID', 'y': 'מספר הודעות'}
)
fig_top_users.show()

In [163]:
df[df['campaign_details'].fillna('0').str.contains('אתה חי?')][['campaign_details','sso_id']]

,campaign_details,sso_id
357,אני מאוד אוהב את ״החשוד המיידי״ . תציע לי סרטי...,3081113310
425,האם אתה חשוף לכל הכתבות בהארץ או רק לתחום הקול...,9037351870
534,אתה חי? בדיקה,8010962861
837,מה אתה חושב,8012159237
1170,מה אתה חושב על מובלנד,167422
2338,<strong>גילי איזיקוביץ</strong> ממליצה על הסדר...,8010940799


In [164]:

# 6. מטריקס זמני - פעילות לפי יום ושעה (Heatmap)
df['date'] = pd.to_datetime(df['event_time']).dt.date
df['day_of_week'] = pd.to_datetime(df['event_time']).dt.day_name()
daily_hourly = df.groupby(['day_of_week', 'hour']).size().reset_index(name='count')

# יצירת heatmap
pivot_table = daily_hourly.pivot(index='day_of_week', columns='hour', values='count').fillna(0)
fig_heatmap = px.imshow(
    pivot_table,
    title="מטריקס פעילות - יום ושעה",
    labels=dict(x="שעה", y="יום בשבוע", color="מספר הודעות"),
    color_continuous_scale="Blues"
)
fig_heatmap.show()

# 7. התפלגות אורך שיחות (מספר הודעות לכל שיחה)
conversation_lengths = df.groupby('id').size()
fig_conv_length = px.histogram(
    x=conversation_lengths.values,
    nbins=20,
    title="התפלגות אורך שיחות (מספר הודעות)",
    labels={'x': 'מספר הודעות בשיחה', 'y': 'מספר שיחות'}
)
fig_conv_length.show()

# 8. יחס שאלות לתשובות
question_count = len(df[df['campaign_name'] == 'open question'])
answer_count = len(df[df['campaign_name'] == 'response answer'])

fig_ratio = px.pie(
    values=[question_count, answer_count],
    names=['שאלות משתמשים', 'תשובות מודל'],
    title="יחס שאלות לתשובות"
)
fig_ratio.show()

# 9. התפלגות זמנית - פעילות לפי תאריך
daily_activity = df.groupby('date').size().reset_index(name='activity')
daily_activity['date'] = pd.to_datetime(daily_activity['date'])

fig_daily = px.line(
    daily_activity,
    x='date',
    y='activity',
    title="פעילות יומית",
    labels={'date': 'תאריך', 'activity': 'מספר הודעות'}
)
fig_daily.show()

# 10. סטטיסטיקות תיאוריות
print("סטטיסטיקות כלליות:")
print(f"מספר כולל של הודעות: {len(df)}")
print(f"מספר שיחות ייחודיות: {df['id'].nunique()}")
print(f"מספר משתמשים ייחודיים: {df['sso_id'].nunique()}")
print(f"אורך שאלה ממוצע: {df[df['campaign_name'] == 'open question']['question_length'].mean():.1f} מילים")
print(f"אורך תשובה ממוצע: {df[df['campaign_name'] == 'response answer']['answer_length'].mean():.1f} מילים")

סטטיסטיקות כלליות:
מספר כולל של הודעות: 2639
מספר שיחות ייחודיות: 596
מספר משתמשים ייחודיים: 458
אורך שאלה ממוצע: 5.8 מילים
אורך תשובה ממוצע: 66.4 מילים


In [165]:
# speed dashboard
df['rag_speed_rounded'] = df['rag_speed'].astype(float).round(1)
df['llm_speed_rounded'] = df['llm_speed'].astype(float).round(1)
df['total_time_rounded'] = df['total_time'].astype(float).round(1)


# Combined speed comparison
from plotly.subplots import make_subplots

fig_combined = make_subplots(
    rows=2, cols=2,
    subplot_titles=('מהירות RAG', 'מהירות LLM', 'זמן כולל', 'השוואת מהירויות'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Add histograms
fig_combined.add_trace(
    go.Histogram(x=df['rag_speed_rounded'].dropna(),
                name='RAG',
                nbinsx=20,
                opacity=0.7,
                marker_color='#1f77b4'),
    row=1, col=1
)

fig_combined.add_trace(
    go.Histogram(x=df['llm_speed_rounded'].dropna(),
                name='LLM',
                nbinsx=20,
                opacity=0.7,
                marker_color='#ff7f0e'),
    row=1, col=2
)

fig_combined.add_trace(
    go.Histogram(x=df['total_time_rounded'].dropna(),
                name='זמן כולל',
                nbinsx=20,
                opacity=0.7,
                marker_color='#2ca02c'),
    row=2, col=1
)

# Box plot comparison
fig_combined.add_trace(
    go.Box(y=df['rag_speed'].astype(float).dropna(),
           name='RAG',
           boxpoints='outliers',
           marker_color='#1f77b4'),
    row=2, col=2
)

fig_combined.add_trace(
    go.Box(y=df['llm_speed'].astype(float).dropna(),
           name='LLM',
           boxpoints='outliers',
           marker_color='#ff7f0e'),
    row=2, col=2
)

fig_combined.add_trace(
    go.Box(y=df['total_time'].astype(float).dropna(),
           name='זמן כולל',
           boxpoints='outliers',
           marker_color='#2ca02c'),
    row=2, col=2
)

fig_combined.update_layout(
    height=800,
    title_text="ניתוח מהירויות מערכת",
    title_x=0.5,
    showlegend=True
)

# Update axes labels
fig_combined.update_xaxes(title_text="שניות", row=1, col=1)
fig_combined.update_xaxes(title_text="שניות", row=1, col=2)
fig_combined.update_xaxes(title_text="שניות", row=2, col=1)
fig_combined.update_yaxes(title_text="תדירות", row=1, col=1)
fig_combined.update_yaxes(title_text="תדירות", row=1, col=2)
fig_combined.update_yaxes(title_text="תדירות", row=2, col=1)
fig_combined.update_yaxes(title_text="שניות", row=2, col=2)

fig_combined.show()

# Print speed statistics
print("סטטיסטיקות מהירות:")
print(f"מהירות RAG ממוצעת: {df['rag_speed'].astype(float).mean():.2f} שניות")
print(f"מהירות LLM ממוצעת: {df['llm_speed'].astype(float).mean():.2f} שניות")
print(f"זמן כולל ממוצע: {df['total_time'].astype(float).mean():.2f} שניות")
print(f"מהירות RAG חציונית: {df['rag_speed'].astype(float).median():.2f} שניות")
print(f"מהירות LLM חציונית: {df['llm_speed'].astype(float).median():.2f} שניות")
print(f"זמן כולל חציוני: {df['total_time'].astype(float).median():.2f} שניות")

סטטיסטיקות מהירות:
מהירות RAG ממוצעת: 0.36 שניות
מהירות LLM ממוצעת: 4.07 שניות
זמן כולל ממוצע: 4.47 שניות
מהירות RAG חציונית: 0.30 שניות
מהירות LLM חציונית: 3.72 שניות
זמן כולל חציוני: 4.09 שניות


In [166]:
# dashboard for function calls: writer_filter, genres, streaming_platforms, media_type

# Filter out only response answer rows (as these contain the function call data)
response_df = df[df['campaign_name'] == 'response answer'].copy()

print(f"Total response messages: {len(response_df)}")

# Data preparation
# 1. Media Type
media_type_counts = response_df['media_type'].value_counts()
media_type_percentages = (media_type_counts / media_type_counts.sum() * 100).round(1)

# 2. Genres - Extract unique genres
all_genres = []
for genres in response_df['genres'].dropna():
    if isinstance(genres, str) and genres.strip():
        if ',' in genres:
            all_genres.extend([g.strip() for g in genres.split(',')])
        else:
            all_genres.append(genres.strip())

genre_counts = pd.Series(all_genres).value_counts().head(10) if all_genres else pd.Series([])
genre_percentages = (genre_counts / genre_counts.sum() * 100).round(1) if len(genre_counts) > 0 else pd.Series([])

# 3. Streaming Platforms - Extract unique platforms
all_platforms = []
for platforms in response_df['streaming_platforms'].dropna():
    if isinstance(platforms, str) and platforms.strip():
        if ',' in platforms:
            all_platforms.extend([p.strip() for p in platforms.split(',')])
        else:
            all_platforms.append(platforms.strip())

platform_counts = pd.Series(all_platforms).value_counts().head(8) if all_platforms else pd.Series([])
platform_percentages = (platform_counts / platform_counts.sum() * 100).round(1) if len(platform_counts) > 0 else pd.Series([])

# 4. Writer Filter
writer_counts = response_df['writer_filter'].value_counts()
writer_counts = writer_counts[writer_counts.index != ''].head(8)
writer_percentages = (writer_counts / writer_counts.sum() * 100).round(1) if len(writer_counts) > 0 else pd.Series([])

# Create subplots - 2x2 grid
fig_categorical = make_subplots(
    rows=2, cols=2,
    subplot_titles=('התפלגות סוגי מדיה (מנורמל)', 'ז\'אנרים פופולריים (מנורמל)', 
                   'פלטפורמות סטרימינג (מנורמל)', 'כותבים/מבקרים פופולריים (מנורמל)'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 1. Media Type - Pie Chart with percentages
if len(media_type_counts) > 0:
    # Create labels with both count and percentage
    labels_with_percent = [f"{label}<br>{count} ({percent}%)" 
                          for label, count, percent in zip(media_type_counts.index, 
                                                          media_type_counts.values, 
                                                          media_type_percentages.values)]
    
    fig_categorical.add_trace(
        go.Pie(
            values=media_type_percentages.values,
            labels=media_type_counts.index,
            name="סוגי מדיה",
            showlegend=True,
            textinfo='label+percent',
            hovertemplate='<b>%{label}</b><br>כמות: %{value}<br>אחוז: %{percent}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Genres - Horizontal Bar Chart with percentages
if len(genre_counts) > 0:
    # Create hover text with both count and percentage
    hover_text = [f"{genre}<br>כמות: {count}<br>אחוז: {percent}%" 
                  for genre, count, percent in zip(genre_counts.index, 
                                                  genre_counts.values, 
                                                  genre_percentages.values)]
    
    fig_categorical.add_trace(
        go.Bar(
            x=genre_percentages.values,
            y=genre_counts.index,
            orientation='h',
            name="ז'אנרים",
            marker_color='lightblue',
            showlegend=False,
            text=[f"{percent}%" for percent in genre_percentages.values],
            textposition='outside',
            hovertext=hover_text,
            hoverinfo='text'
        ),
        row=1, col=2
    )

# 3. Streaming Platforms - Vertical Bar Chart with percentages
if len(platform_counts) > 0:
    hover_text = [f"{platform}<br>כמות: {count}<br>אחוז: {percent}%" 
                  for platform, count, percent in zip(platform_counts.index, 
                                                      platform_counts.values, 
                                                      platform_percentages.values)]
    
    fig_categorical.add_trace(
        go.Bar(
            x=platform_counts.index,
            y=platform_percentages.values,
            name="פלטפורמות",
            marker_color='lightgreen',
            showlegend=False,
            text=[f"{percent}%" for percent in platform_percentages.values],
            textposition='outside',
            hovertext=hover_text,
            hoverinfo='text'
        ),
        row=2, col=1
    )

# 4. Writers - Horizontal Bar Chart with percentages
if len(writer_counts) > 0:
    hover_text = [f"{writer}<br>כמות: {count}<br>אחוז: {percent}%" 
                  for writer, count, percent in zip(writer_counts.index, 
                                                    writer_counts.values, 
                                                    writer_percentages.values)]
    
    fig_categorical.add_trace(
        go.Bar(
            x=writer_percentages.values,
            y=writer_counts.index,
            orientation='h',
            name="כותבים",
            marker_color='lightsalmon',
            showlegend=False,
            text=[f"{percent}%" for percent in writer_percentages.values],
            textposition='outside',
            hovertext=hover_text,
            hoverinfo='text'
        ),
        row=2, col=2
    )

# Update layout
fig_categorical.update_layout(
    height=800,
    title_text="ניתוח משתנים קטגוריאליים (מנורמל באחוזים)",
    title_x=0.5,
    font=dict(size=12)
)

# Update x and y axes labels
fig_categorical.update_xaxes(title_text="אחוז (%)", row=1, col=2)
fig_categorical.update_yaxes(title_text="ז'אנר", row=1, col=2)

fig_categorical.update_xaxes(title_text="פלטפורמה", row=2, col=1, tickangle=45)
fig_categorical.update_yaxes(title_text="אחוז (%)", row=2, col=1)

fig_categorical.update_xaxes(title_text="אחוז (%)", row=2, col=2)
fig_categorical.update_yaxes(title_text="כותב/מבקר", row=2, col=2)

fig_categorical.show()

# Create a more readable Genre vs Media Type analysis with normalization
if all_genres:
    genre_media_df = response_df[response_df['genres'].notna() & response_df['media_type'].notna()]
    
    if len(genre_media_df) > 0:
        # Create a more detailed genre-media analysis
        genre_media_analysis = []
        for _, row in genre_media_df.iterrows():
            if isinstance(row['genres'], str) and row['genres'].strip():
                genres_list = [g.strip() for g in row['genres'].split(',')]
                for genre in genres_list:
                    genre_media_analysis.append({
                        'genre': genre,
                        'media_type': row['media_type']
                    })
        
        if genre_media_analysis:
            genre_media_detail_df = pd.DataFrame(genre_media_analysis)
            
            # Select top 8 genres for better visualization
            top_genres = genre_media_detail_df['genre'].value_counts().head(8).index
            filtered_df = genre_media_detail_df[genre_media_detail_df['genre'].isin(top_genres)]
            
            # Create grouped bar chart with normalization
            fig_genre_media = go.Figure()
            
            # Chart: For each genre, show breakdown by media type
            genre_media_counts = filtered_df.groupby(['genre', 'media_type']).size().reset_index(name='count')
            
            # Get unique media types for consistent coloring
            unique_media_types = genre_media_counts['media_type'].unique()
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
            
            for i, media_type in enumerate(unique_media_types):
                data = genre_media_counts[genre_media_counts['media_type'] == media_type]
                fig_genre_media.add_trace(
                    go.Bar(
                        x=data['genre'],
                        y=data['count'],
                        name=f'{media_type}',
                        marker_color=colors[i % len(colors)],
                        text=data['count'],
                        textposition='outside',
                        hovertemplate=f'<b>{media_type}</b><br>ז\'אנר: %{{x}}<br>כמות: %{{y}}<extra></extra>'
                    )
                )
            
            fig_genre_media.update_layout(
                height=500,
                title_text="התפלגות ז'אנרים לפי סוג מדיה",
                title_x=0.5,
                barmode='group',  # Changed to group for side-by-side bars
                font=dict(size=11),
                xaxis_title="ז'אנר",
                yaxis_title="כמות",
                xaxis_tickangle=45
            )
            
            fig_genre_media.show()
            
            # Create a summary table
            print("\nטבלת סיכום - ז'אנרים לפי סוג מדיה:")
            genre_media_crosstab = pd.crosstab(
                filtered_df['genre'], 
                filtered_df['media_type'], 
                margins=True
            )
            print(genre_media_crosstab)

# Print summary statistics with normalization
print("\nסטטיסטיקות קטגוריאליות (מנורמל):")
print(f"סוגי מדיה שונים: {len(media_type_counts)}")
print(f"ז'אנרים ייחודיים: {len(set(all_genres))}")
print(f"פלטפורמות סטרימינג: {len(set(all_platforms))}")
print(f"כותבים/מבקרים: {len(writer_counts)}")

if len(media_type_counts) > 0:
    print(f"סוג המדיה הפופולרי ביותר: {media_type_counts.index[0]} ({media_type_percentages.iloc[0]}% - {media_type_counts.iloc[0]} פעמים)")
if len(genre_counts) > 0:
    print(f"הז'אנר הפופולרי ביותר: {genre_counts.index[0]} ({genre_percentages.iloc[0]}% - {genre_counts.iloc[0]} פעמים)")
if len(platform_counts) > 0:
    print(f"פלטפורמת הסטרימינג הפופולרית: {platform_counts.index[0]} ({platform_percentages.iloc[0]}% - {platform_counts.iloc[0]} פעמים)")
if len(writer_counts) > 0:
    print(f"הכותב/מבקר הפופולרי ביותר: {writer_counts.index[0]} ({writer_percentages.iloc[0]}% - {writer_counts.iloc[0]} פעמים)")




Total response messages: 1252



טבלת סיכום - ז'אנרים לפי סוג מדיה:
media_type       movie  series  All
genre                              
אקשן          7     13      25   45
דוקומנטרי    11      7      15   33
דרמה         38     23     154  215
מדע בדיוני   14      7      26   47
מותחן        26      4     104  134
פשע          16      2      68   86
קומדיה       40     16      82  138
רומנטיקה     17      2      15   34
All         169     74     489  732

סטטיסטיקות קטגוריאליות (מנורמל):
סוגי מדיה שונים: 3
ז'אנרים ייחודיים: 33
פלטפורמות סטרימינג: 15
כותבים/מבקרים: 6
סוג המדיה הפופולרי ביותר:  (46.9% - 583 פעמים)
הז'אנר הפופולרי ביותר: דרמה (27.5% - 215 פעמים)
פלטפורמת הסטרימינג הפופולרית: Netflix (49.0% - 276 פעמים)
הכותב/מבקר הפופולרי ביותר: אורון שמיר (36.4% - 4 פעמים)


In [167]:
df[df['regenerate'] == 'true']

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,session_id,total_time,rag_speed,...,streaming_platforms,id,question_length,answer_length,hour,date,day_of_week,rag_speed_rounded,llm_speed_rounded,total_time_rounded
1393,456438023110,"בהחלט! אם אתם מחפשים משהו קצת פחות קליל, <stro...",response answer,App,2025-07-01 13:47:56,impression,14,0ec2b778-9ba2-4990-9b7e-3ccc6e5a4a05,6.691929340362549,0,...,,456438023110_0ec2b778-9ba2-4990-9b7e-3ccc6e5a4a05,NaN,50.0,13,2025-07-01,Tuesday,0.0,6.7,6.7
1530,7616696257,"סליחה, איני יכול לעזור עם בקשה זו. האם תרצו שא...",response answer,Mobile,2025-07-01 13:11:00,impression,402,35195e46-1cb7-4a18-9621-3cd242a63fc8,1.3858659267425537,0,...,,7616696257_35195e46-1cb7-4a18-9621-3cd242a63fc8,NaN,26.0,13,2025-07-01,Tuesday,0.0,1.4,1.4
1572,1956116702,"אני מבין, סליחה על הטעות. אתם צודקים, התבקשתי ...",response answer,Mobile,2025-07-01 12:52:20,impression,527,fbc1ccec-9463-4956-aac3-da0b17142089,2.8045670986175537,0,...,,1956116702_fbc1ccec-9463-4956-aac3-da0b17142089,NaN,33.0,12,2025-07-01,Tuesday,0.0,2.8,2.8
1938,9037484962,"בטח, אם אתם מחפשים סדרת <strong>feel good</str...",response answer,App,2025-07-01 11:10:25,impression,1517,0c9c9fce-888f-4b05-a853-cddeadaca8f0,6.062093019485474,0.2955935001373291,...,Netflix,9037484962_0c9c9fce-888f-4b05-a853-cddeadaca8f0,NaN,64.0,11,2025-07-01,Tuesday,0.3,5.8,6.1
2023,8013497274,"אני מבין שאתם מחפשים משהו קצר במיוחד! עם זאת, ...",response answer,App,2025-07-01 10:46:57,impression,1755,c9b53995-c549-484b-8ba5-43cf3ea9a7a4,6.234783887863159,0,...,,8013497274_c9b53995-c549-484b-8ba5-43cf3ea9a7a4,NaN,99.0,10,2025-07-01,Tuesday,0.0,6.2,6.2
2055,8010962763,"בהחלט! יש לי המלצה מצוינת עבורכם: <strong>""סעי...",response answer,Desktop,2025-07-01 10:35:31,impression,1838,2f4e95b0-23bd-4859-a02a-50925cb125c0,4.741535663604736,0.30356264114379883,...,,8010962763_2f4e95b0-23bd-4859-a02a-50925cb125c0,NaN,15.0,10,2025-07-01,Tuesday,0.3,4.4,4.7
2203,6503516536,None,response answer,App,2025-07-01 09:45:58,impression,2248,ed80c10c-19bf-4aef-8f98-4cd9edb4ab63,3.031012773513794,0.2948453426361084,...,,6503516536_ed80c10c-19bf-4aef-8f98-4cd9edb4ab63,NaN,NaN,9,2025-07-01,Tuesday,0.3,2.7,3.0
2419,8011691191,בהחלט! <strong>גילי איזיקוביץ'</strong> ממליצה...,response answer,App,2025-07-01 08:25:51,impression,2854,f57c345d-03c1-45b8-b7ec-7608a9168b92,3.73488712310791,0.32749199867248535,...,,8011691191_f57c345d-03c1-45b8-b7ec-7608a9168b92,NaN,128.0,8,2025-07-01,Tuesday,0.3,3.4,3.7
2420,8013505454,"שלום! כאן רני, מבקר הקולנוע והטלוויזיה של הארץ...",response answer,Desktop,2025-07-01 08:25:39,impression,2855,69dcf3a4-73b9-425e-bec7-27edfe20ac25,1.893188238143921,0,...,,8013505454_69dcf3a4-73b9-425e-bec7-27edfe20ac25,NaN,36.0,8,2025-07-01,Tuesday,0.0,1.9,1.9
2574,50000175162,שלום! מה תרצו לראות היום? 🍿 אולי נתחיל עם איז...,response answer,App,2025-07-01 07:23:36,impression,3288,6a5939bf-33e6-4429-8c0c-b9dcd6b3163d,0.998405933380127,0,...,,50000175162_6a5939bf-33e6-4429-8c0c-b9dcd6b3163d,NaN,22.0,7,2025-07-01,Tuesday,0.0,1.0,1.0


In [169]:
df['error'].value_counts().sort_index()

error
0    1243
Name: count, dtype: int64

In [170]:
df.shape

(2639, 37)

In [171]:
2639/2

1319.5